In [1]:
import pandas as pd
from Bio import SeqIO
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import pearsonr, spearmanr, fisher_exact, chi2_contingency
import math
import gzip
import pickle
import os
import gzip

In [11]:
library=pd.read_csv('./dataframes/library_varseq_FINAL.csv', index_col='Unnamed: 0')

libdfbarcodes=library.barcode.to_dict()
readcounts={idx:0 for idx in libdfbarcodes.keys()}
readids={idx:[] for idx in libdfbarcodes.keys()}

In [36]:
def map_barcodes(libdfbarcodes, fastqr2, outfolder, barcodelength=None):
    if isinstance(libdfbarcodes, pd.Series):
        libbcdict=libdfbarcodes.to_dict()
    elif type(libdfbarcodes)==dict:
        libbcdict=libdfbarcodes
    else:
        print('The input is neither a dict nor a pandas Series.')
        exit()
        
    if os.path.isdir(outfolder)==False:
        os.mkdir(outfolder)
        
    samplename=fastqr2.split('/')[-1].split('.')[0]
    
    bcindexdict={value[:barcodelength]:key for key, value in libbcdict.items()}

    readcounts={idx:0 for idx in libdfbarcodes.keys()}
    readids={idx:[] for idx in libdfbarcodes.keys()}
    
    readcount=0
    unmappedreadcount=0
    unmappedreadseqs=[]
    
    with gzip.open(fastqr2, "rt") as handle:
        for record in SeqIO.parse(handle, 'fastq'):
            readcount+=1
            primerstart=record.seq.find('GAGCGCACCCGTCCGAGC')
            if primerstart==-1:
                unmappedreadcount+=1
                unmappedreadseqs.append(record.seq)
                pass
            else:
                if barcodelength>0:
                    readbc=record.seq[primerstart+18+12+16:primerstart+18+12+16+barcodelength]
                    if len(readbc)<barcodelength:
                        unmappedreadcount+=1
                        unmappedreadseqs.append(record.seq)
                        pass

                else:
                    readbc=record.seq[primerstart+18+12+16:primerstart+18+12+16+14]
                    if len(readbc)<14:
                        unmappedreadcount+=1
                        unmappedreadseqs.append(record.seq)
                        pass

                if readbc in bcindexdict.keys():
                    readcounts[bcindexdict[readbc]]+=1
                    readids[bcindexdict[readbc]].append(record.id)
                else:
                    unmappedreadcount+=1
                    unmappedreadseqs.append(record.seq)
                    pass

 #   print(str(unmappedreadcount) + ' reads out of '+str(readcount)+' could not be mapped.')
    
    with open(outfolder+'/'+samplename+'__readcounts.pkl', 'wb') as out:
        pickle.dump(readcounts, out, protocol=pickle.HIGHEST_PROTOCOL)
    with open(outfolder+'/'+samplename+'__readids.pkl', 'wb') as out:
        pickle.dump(readids, out, protocol=pickle.HIGHEST_PROTOCOL)

    with open(outfolder+'/'+samplename+'__unmappedreadseqs.pkl', 'wb') as out:
        pickle.dump(unmappedreadseqs, out, protocol=pickle.HIGHEST_PROTOCOL)

    
    return readcount, unmappedreadcount



In [41]:
readcount, unmappedreadcount=map_barcodes(libdfbarcodes, 
                                          './example_data/examplefastqfile_R2_001.fastq.gz', 
                                          './dataframes/barcodemapping_out', 
                                          barcodelength=12)


In [42]:
print('total number of reads: ' + str(readcount) +
      '\nnumber of unmapped reads: ' + str(unmappedreadcount) + 
      '\npercentage of unmapped reads: ' + str((unmappedreadcount/readcount)*100))

total number of reads: 250
number of unmapped reads: 57
percentage of unmapped reads: 22.8


In [46]:
def get_umis(readids, r1fastq, rtprimer):
    umis={idx:[] for idx in readids.keys()}
    
    readids_inv={}
    for key in readids.keys():
        val=readids[key]
        if type(val)!=int:
            readids_inv.update({valn:key for valn in val})
        
    with gzip.open(r1fastq, "rt") as f:
        for record in SeqIO.parse(f, 'fastq'):
            if (record.id in readids_inv.keys())&(record.id!=0):
                if rtprimer=='dt':
                    umis[readids_inv[record.id]].append(str(record.seq[6:18]))
                elif rtprimer=='utr1':
                    startpos=record.seq.find('CAAACAACA')
                    if startpos>9:
                        umis[readids_inv[record.id]].append(str(record.seq[1:10]))
                    
    umisdf=pd.DataFrame()
    umiss=pd.Series(umis)
    umiss=umiss.drop([x for x in umiss.index if type(x)!=int])
    umisdf['all_umis']=umiss
    umisdf['n_all_umis']=umisdf.all_umis.apply(lambda x: len(x))
    umisdf['n_unique_umis']=umisdf.all_umis.apply(lambda x: len(pd.Series(x).unique()))
    umisdf.to_pickle('./dataframes/barcodemapping_out/'+r1fastq.split('/')[-1].split('.')[0]+'_umis.pkl')
    return umisdf.n_all_umis.sum(), umisdf.n_unique_umis.sum()


In [47]:
for filename in os.listdir('./dataframes/barcodemapping_out'):
    if ('__readids' in filename):
        with open('./dataframes/barcodemapping_out/'+filename, 'rb') as out:
            readids=pickle.load(out)

        n_all_umis, n_unique_umis = get_umis(readids, 
            './example_data/'+filename.split('R2')[0]+'R1_001.fastq.gz', 
                         'dt')
        with open('./dataframes/barcodemapping_out/umicount_summary.csv', 'a') as summaryout:
            summaryout.write(filename+','+str(n_all_umis)+','+str(n_unique_umis)+'\n')
            


/Users/martin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [48]:
allcounts_example=pd.DataFrame()
umicounts_example=pd.DataFrame()

for filename in os.listdir('./dataframes/barcodemapping_out'):
    if ('_umis.pkl' in filename):
        df_to_append=pd.read_pickle('./dataframes/barcodemapping_out/'+filename)

        colname=filename.split('_')[0]

        if colname in umicounts_example.columns:
            umicounts_CADdeg[colname]=umicounts_CADdeg[colname]+df_to_append['n_unique_umis']
            umicounts_example[colname]=allcounts_CADdeg[colname]+df_to_append['n_all_umis']
        else:
            umicounts_example[colname]=df_to_append['n_unique_umis']
            umicounts_example.rename(columns={'n_unique_umis':colname}, inplace=True)
            allcounts_example[colname]=df_to_append['n_all_umis']
            allcounts_example.rename(columns={'n_all_umis':colname}, inplace=True)
            

In [50]:
umicounts_example[umicounts_example.examplefastqfile>0]

,examplefastqfile
10,1
33,2
34,1
150,1
420,1
...,...
13241,1
13344,1
13353,1
13657,1


In [51]:
allcounts_example.to_pickle('./dataframes/allcounts_example.pkl')
umicounts_example.to_pickle('./dataframes/umicounts_example.pkl')
